In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
week12_link = 'http://www.footballlocks.com/nfl_odds_week_12.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week12_html = requests.get(week12_link).text

soup = BeautifulSoup(week12_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[274])

<tr>
<td>11/23 12:30 ET</td>
<td>Minnesota</td>
<td>-2.5</td>
<td>At Detroit</td>
<td>46</td>
<td>-$145 +$125</td>
</tr>
<tr>
<td>11/27 8:30 ET</td>
<td>At Seattle</td>
<td>-10</td>
<td>Green Bay</td>
<td>42</td>
<td>-$575 +$415</td>
</tr>


In [3]:
#want to 275 because it does not include last one
table = tr[51:275]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week12_df = pd.DataFrame(string_table)

week12_df.columns = ['all_data']

week12_df.head()

,all_data
0,<tr>\n<td>11/23 12:30 ET</td>\n<td>Minnesota</...
1,<tr>\n<td>11/23 4:30 ET</td>\n<td>LA Chargers<...
2,<tr>\n<td>11/23 8:30 ET</td>\n<td>At Washingto...
3,<tr>\n<td>11/26 1:00 ET</td>\n<td>At Cincinnat...
4,<tr>\n<td>11/26 1:00 ET</td>\n<td>At Philadelp...


In [4]:
#want to clean the date data

dates = [x.split('td')[1] for x in week12_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week12_df['date'] = cleaned_dates

In [5]:
#pull out favorites

week12_df['cleaned_favorites'] = week12_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week12_df['cleaned_underdogs'] = week12_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week12_df['cleaned_spread'] = week12_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week12_df['cleaned_over_under'] = week12_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week12_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week12_df['money_odds'] = money_odds

#input week number for each game
week12_df['week_number'] = '12'

In [6]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2017
year_list = []
for x in week12_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week12_df['year'] = year_list

In [7]:
#create a final dataframe for week 12
week12_final = week12_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week12_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,11/23 12:30 ET,2017,12,Minnesota,At Detroit,-2.5,46,-$145 +$125
1,11/23 4:30 ET,2017,12,LA Chargers,At Dallas,-1.5,46.5,-$125 +$105
2,11/23 8:30 ET,2017,12,At Washington,NY Giants,-7,44,-$340 +$280
3,11/26 1:00 ET,2017,12,At Cincinnati,Cleveland,-7.5,39,-$370 +$300
4,11/26 1:00 ET,2017,12,At Philadelphia,Chicago,-13.5,43.5,-$1100 +$700


In [8]:
#week12_final to pickle
week12_final.to_pickle('week12_final.pkl')